In [2]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Model

model = ResNet50(weights='imagenet', include_top=False)
# model = Model(inputs=model.input, outputs=model.get_layer(10).output)
model.summary()

img_path = 'labeledYelp/_8B40xRtabQop1QXbpGTtA.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_data = image.img_to_array(img)
img_data = np.expand_dims(img_data, axis=0)
img_data = preprocess_input(img_data)

resnet_feature = model.predict(img_data)
print(resnet_feature.shape)


/home/nihira/anaconda2/envs/py36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

(1, 7, 7, 2048)


In [3]:
import glob
gl = glob.glob("/home/nihira/Documents/CS6220BigData/Project/neural-image-assessment-master/labeledYelp/*.jpg")


In [4]:
len(gl)

483

In [5]:
gl[0]

'/home/nihira/Documents/CS6220BigData/Project/neural-image-assessment-master/labeledYelp/1d_XuE0VXZrk7Lg2rWJPSA.jpg'

In [6]:
filenames = []
for glo in gl:
    filenames.append(glo.split("/")[8].split(".")[0])

In [7]:
vgg16_feature_list = []
# subdir = "labelledYelp/"
for img_path in gl:        # process the files under the directory 'dogs' or 'cats'
        # ...

        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list.append(vgg16_feature_np.flatten())
        
vgg16_feature_list_np = np.array(vgg16_feature_list)
# kmeans = KMeans(n_clusters=2, random_state=0).fit(vgg16_feature_list_np)


In [8]:
vgg16_feature_list_np.shape


(483, 100352)

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv("../FinalLabels/Final_Test_Labels.csv")

In [11]:
df.shape

(483, 4)

In [12]:
list(df)

['photo_id', 'finalLabel', 'value', 'label']

In [13]:
df.dtypes

photo_id       object
finalLabel      int64
value         float64
label           int64
dtype: object

In [14]:
scores = []
labels = []
categories = []
for file in filenames:
#     print("Entered")
    f = df.loc[df['photo_id'] == file]
    if len(f)>0:
#         print(f.value.item())
#     print(f.value.item())
        scores.append(f.value.item())
        labels.append(f.finalLabel.item())
        categories.append(f.label.item())
    else:
        print(file)

In [15]:
len(scores)

483

In [16]:
scores[0]

4.85133

In [17]:
scores = np.array(scores)
labels = np.array(labels)
categories = np.array(categories)

In [18]:
labels.shape

(483,)

In [19]:
vgg16_feature_list_np.shape

(483, 100352)

In [21]:
from sklearn.decomposition import PCA
myPca = PCA(n_components=200)
vgg16_feature_list_np = myPca.fit_transform(vgg16_feature_list_np)

MemoryError: 

In [20]:
# all_data = np.append(vgg16_feature_list_np, scores, axis = 1)
all_data = np.column_stack([vgg16_feature_list_np,scores, categories])


MemoryError: 

In [42]:
all_data.shape

(483, 25090)

In [112]:
# all_data = np.column_stack([all_data,labels])


In [113]:
# all_data.shape

(483, 25090)

In [57]:
from sklearn.svm import SVC

In [117]:
clf = SVC(gamma='auto')

In [118]:
clf.fit(all_data, labels) 


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [119]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(all_data)
accuracy_score(labels, y_pred)
precision_score(labels, y_pred1)

1.0

In [43]:
import pandas as pd
df1 = pd.read_csv("../FinalLabels/Final_Train_Labels.csv")

In [44]:
list(df1)

['photo_id',
 'nima',
 'nimaMetrics',
 'clarifai',
 'nimaClarifai',
 'nimaMetricsClarifai',
 'value',
 'label']

In [45]:
df1.head()

,photo_id,nima,nimaMetrics,clarifai,nimaClarifai,nimaMetricsClarifai,value,label
0,LM8N-DdGwgz8sGBuVkZ9uw,1,1,1,1,1,5.94856,food
1,b0dJWV7OcGkNEjWw3rOqsA,1,1,0,0,0,5.94517,inside
2,qPfWXx8xOSp44sMhc-Rbfg,1,1,0,0,0,5.86847,inside
3,06vyJxHn0EvRiiHJlWsx0g,1,1,1,1,1,5.85913,food
4,XZWmzEMW6aQsXg9FmXBZcw,1,1,0,0,0,5.83328,inside


In [46]:
import glob
gl1 = glob.glob("/home/nihira/Documents/CS6220BigData/Project/neural-image-assessment-master/trainingYelp/*.jpg")


In [47]:
len(gl1)

2795

In [48]:
gl1[0]

'/home/nihira/Documents/CS6220BigData/Project/neural-image-assessment-master/trainingYelp/d3Xl5TN6Wq6OapbNO4Qmqg.jpg'

In [49]:
filenames1 = []
for glo in gl1:
    filenames1.append(glo.split("/")[8].split(".")[0])

In [50]:
len(filenames1)

2795

In [51]:
vgg16_feature_list1 = []
# subdir = "labelledYelp/"
for img_path in gl1:        # process the files under the directory 'dogs' or 'cats'
        # ...

        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list1.append(vgg16_feature_np.flatten())
        
vgg16_feature_list_np1 = np.array(vgg16_feature_list1)
# kmeans = KMeans(n_clusters=2, random_state=0).fit(vgg16_feature_list_np)


In [52]:
vgg16_feature_list_np1.shape


(2795, 25088)

In [53]:
scores1 = []
labels1 = []
category = []
for file in filenames1:
    f = df1.loc[df1['photo_id'] == file]
    scores1.append(f.value.item())
    labels1.append(f.nimaMetricsClarifai.item())
    category.append(f.clarifai.item())

     

In [54]:
scores1 = np.array(scores1)
labels1 = np.array(labels1)
category = np.array(category)

In [55]:
# all_data = np.append(vgg16_feature_list_np, scores, axis = 1)
all_data1 = np.column_stack([vgg16_feature_list_np1,scores1,category])


In [58]:
clf1 = SVC(gamma='auto')
clf1.fit(all_data1, labels1) 


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [59]:
from sklearn.metrics import accuracy_score, precision_score
y_pred1 = clf1.predict(all_data)
accuracy_score(labels, y_pred1)

0.6728778467908902

In [60]:
from sklearn.metrics import accuracy_score, precision_score


In [73]:
precision_score(labels, y_pred1)

0.6742738589211619

In [62]:
sum(labels==y_pred1)

325

In [63]:
len(labels)

483

In [64]:
sum(labels==1)

326

In [68]:
sum(y_pred1==1)

482

In [69]:
sum(y_pred1==0)

1

In [70]:
sum(labels1==1)

2283

In [71]:
sum(labels1==0)

512